In [1]:
import torch
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from scipy.spatial.distance import cosine
import librosa
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

c:\Tyan\wav2vec\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PhonemeExtractor:
    """Class để load model một lần và tái sử dụng"""
    
    def __init__(self, model_name: str = "bookbot/wav2vec2-ljspeech-gruut", device: str = None):
        """
        Initialize model and processor once
        
        Args:
            model_name: HuggingFace model name
            device: 'cuda' or 'cpu', auto-detect if None
        """
        print(f"Loading model {model_name}...")
        
        # Auto-detect device
        if device is None:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device(device)
        
        # Load processor và model một lần
        self.processor = Wav2Vec2Processor.from_pretrained(model_name)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()
        
        print(f"Model loaded on {self.device}")
    
    def load_audio(self, audio_path: str, sr: int = 16000) -> np.ndarray:
        """Load audio file and resample to 16kHz."""
        audio, _ = librosa.load(audio_path, sr=sr)
        return audio
    
    def get_phoneme_embeddings(self, audio: np.ndarray) -> Tuple[torch.Tensor, str]:
        """Extract phoneme-level embeddings and predicted phonemes from audio."""
        # Prepare input
        inputs = self.processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
        
        # Move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            # Get hidden states (embeddings)
            outputs = self.model(**inputs, output_hidden_states=False)
            # hidden_states = outputs.hidden_states[-1]  # Last layer
            
            # Get phoneme predictions
            logits = outputs.logits
            predicted_ids = torch.argmax(logits, dim=-1)
            # print(f"Predicted IDs: {predicted_ids}")
            phonemes = self.processor.batch_decode(predicted_ids)[0]
        torch.cuda.empty_cache()
        return phonemes
    
    def process_file(self, audio_path: str) -> Tuple[torch.Tensor, str]:
        """Load audio và extract phonemes trong một lần gọi"""
        audio = self.load_audio(audio_path)
        return self.get_phoneme_embeddings(audio)

In [3]:
# Cell 1: Load model một lần duy nhất (chỉ chạy 1 lần)
extractor = PhonemeExtractor(device='cuda')  # hoặc 'cuda' nếu có GPU

Loading model bookbot/wav2vec2-ljspeech-gruut...
Model loaded on cuda


In [4]:
# Cell 2: Sử dụng nhiều lần mà không cần load lại
ref_phonemes = extractor.process_file("./audio_files/lj_01.wav")
print(ref_phonemes)
# print(ref_embeddings.shape)

p ɹ ɪ n t ɪ ŋ  ɪ n ð əoʊn l i s ɛ n s w ɪ θ w ɪt͡ʃw i ɑ ɹ æ t p ɹ ɛ z ə n t k ə n s ɚ n d  d ɪ f ɚ z f ɹ ʌ m moʊs t ɪ f n ɑ t f ɹ ʌ m ɔ l ð ə ɑ ɹ t s æ n d k ɹ æ f t s ɹ ɛ p ɹ ɪ z ɛ n t ɪ d ɪ n ð ə ɛ k s ɪ b ɪ ʃ ə n


In [5]:
text = "d͡ʒæ n j u ɛ ɹ i "
phonemes = text.strip().split()
print(phonemes)

['d͡ʒæ', 'n', 'j', 'u', 'ɛ', 'ɹ', 'i']


In [7]:
import regex

ipa_string = "d͡ʒæ n j u ɛ ɹ i"

# \X khớp với một cụm grapheme mở rộng (ví dụ: 'd' + '͡' + 'ʒ' sẽ được coi là một)
# Chúng ta cũng lọc bỏ các khoảng trắng
phoneme_letters = [g for g in regex.findall(r'\X', ipa_string) if not g.isspace()]

print(phoneme_letters)

['d͡', 'ʒ', 'æ', 'n', 'j', 'u', 'ɛ', 'ɹ', 'i']
